In [1]:
%reload_ext autoreload
%autoreload 2

import os
import re
import sys
import json
import gc
import warnings
from datetime import datetime, date, timedelta

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import turbodbc
from turbodbc import connect

from IPython.display import JSON
from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

import config
from normalize_funcs import *
from db_funcs import createDBTable, downloadTableToDB, get_mssql_table
 

db_name = config.db_name
# ссылка на гугл csv Словарь чистки объявлений
full_cleaning_link = config.full_cleaning_link
# ссылка на гугл csv Словарь дисконтов по Типам медиа
discounts_link = config.discounts_link


Запрошены записи: 0 - 314
Всего найдено записей: 314

Запрошены записи: 0 - 314
Всего найдено записей: 314



In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

In [3]:
# функция забирает гугл докс с чисткой Объявлений
# можем передать в нее список с названиями типов медиа, по которым отфильтровать датаФрейм
# по умолчанию забираем все
def get_cleaning_dict(media_type_lst=None):
    # т.к. забираем csv по ссылке, чтобы исключить ошибки при добавлении новых столбцов
    # формируем список из номеров колонок от 0 до 23
    cols_count = [i for i in range(23)]
    # опускаем заголовки на 1-ую строку датаФрейма и передаем номера столбцов, которые нам нужны
    df = pd.read_csv(full_cleaning_link, header=None, usecols=cols_count)
    # поднимаем из 1-ой строки названия в заголовки
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    df['media_type'] = df['media_type'].str.upper()
    # если в функцию передан список типов медиа, то оставляем только нужные строки
    if media_type_lst:
        media_type_lst = [i.upper() for i in media_type_lst]
        df = df.query('media_type in @media_type_lst')
# удаляем дубликаты
    df = df.drop_duplicates('media_key_id')
# приводим формат даты к нормальному виду
    df['first_issue_date'] = pd.to_datetime(df['first_issue_date'])
# создаем флаг для очищенных / удаленных объявлений
    df['cleaning_flag'] = df['include_exclude'].apply(lambda x: 1 if x=='include' else 0)
    df = df.drop(['ad_transcribtion'], axis=1)
# приводим строки в верхний регистр, нормализуем цифры и тд.
    custom_ad_dict_int_lst = config.custom_ad_dict_int_lst
    df = normalize_columns_types(df, custom_ad_dict_int_lst)

    return df

In [4]:
# создаем функцию для получения Дисконтов по типам медиа
# можем передать в нее список с названиями типов медиа, по которым отфильтровать датаФрейм
# по умолчанию забираем все

def get_media_discounts(media_type_lst=None):
    cols_count = [i for i in range(3)]
    # опускаем заголовки на 1-ую строку датаФрейма и передаем номера столбцов, которые нам нужны
    df = pd.read_csv(discounts_link, header=None, usecols=cols_count)
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    df['media_type'] = df['media_type'].str.upper()
    
    if media_type_lst:
        media_type_lst = [i.upper() for i in media_type_lst]
        df = df.query('media_type in @media_type_lst')
        
    df = normalize_columns_types(df, ['year'], ['disc'])
    return df

In [5]:
# # dicts_lst = config.nat_tv_slices
# # Список параметров словарей ТВ Индекс для создания таблиц в БД и нормализации данных
# # Название таблицы / Список названий полей  в БД и типы данных / Список целочисденных полей
# tv_index_dicts = {
#     'advertiserListId': [config.tv_index_advertiser_list_dict, config.tv_index_advertiser_list_dict_vars_list, config.tv_index_advertiser_list_dict_int_lst],
#     'brandListId': [config.tv_index_brand_list_dict, config.tv_index_brand_list_dict_vars_list, config.tv_index_brand_list_dict_int_lst],
#     'subbrandListId': [config.tv_index_subbrand_list_dict, config.tv_index_subbrand_list_dict_vars_list, config.tv_index_subbrand_list_dict_int_lst],
#     'modelListId': [config.tv_index_model_list_dict, config.tv_index_model_list_dict_vars_list, config.tv_index_model_list_dict_int_lst],
#     'articleList2Id': [config.tv_index_article_list2_dict, config.tv_index_article_list2_dict_vars_list, config.tv_index_article_list2_dict_int_lst],
#     'articleList3Id': [config.tv_index_article_list3_dict, config.tv_index_article_list3_dict_vars_list, config.tv_index_article_list3_dict_int_lst],
#     'articleList4Id': [config.tv_index_article_list4_dict, config.tv_index_article_list4_dict_vars_list, config.tv_index_article_list4_dict_int_lst],
#     'adSloganAudioId': [config.tv_index_audio_slogan_dict, config.tv_index_audio_slogan_dict_vars_list, config.tv_index_audio_slogan_dict_int_lst],
#     'adSloganVideoId': [config.tv_index_video_slogan_dict, config.tv_index_video_slogan_dict_vars_list, config.tv_index_video_slogan_dict_int_lst],
#     'regionId': [config.tv_index_region_dict, config.tv_index_region_dict_vars_list, config.tv_index_region_dict_int_lst],
#     'tvNetId': [config.tv_index_tv_net_dict, config.tv_index_tv_net_dict_vars_list, config.tv_index_tv_net_dict_int_lst],
#     'tvCompanyId': [config.tv_index_tv_company_dict, config.tv_index_tv_company_dict_vars_list, config.tv_index_tv_company_dict_int_lst],
#     'adTypeId': [config.tv_index_ad_type_dict, config.tv_index_ad_type_dict_vars_list, config.tv_index_ad_type_dict_int_lst],
# }

In [6]:
def get_tv_index_dicts(dict_name, search_lst=None):
    if search_lst:
        search_lst = [str(id) for id in search_lst]
        
    if 'advertiserList' in dict_name:
        df = cats.get_tv_advertiser_list(search_lst)
        df = df.rename(columns={'id': 'advertiserListId', 'name': 'advertiserListName', 'ename': 'advertiserListEName'})

    if 'brandList' in dict_name:
        df = cats.get_tv_brand_list(search_lst)
        df = df.rename(columns={'id': 'brandListId', 'name': 'brandListName', 'ename': 'brandListEName'})
        
    if 'subbrandList' in dict_name:
        df = cats.get_tv_subbrand_list(search_lst)
        df = df.rename(columns={'id': 'subbrandListId', 'name': 'subbrandListName', 'ename': 'subbrandListEName'})

    if 'modelList'in dict_name:
        df = cats.get_tv_model_list(search_lst)
        df = df.rename(columns={'id': 'modelListId', 'name': 'modelListName', 'ename': 'modelListEName'})
        
    if 'articleList2' in dict_name:
        df = cats.get_tv_article_list2(search_lst)
        df = df.rename(columns={'id': 'articleList2Id', 'name': 'articleList2Name', 'ename': 'articleList2EName'})

    if 'articleList3' in dict_name:
        df = cats.get_tv_article_list3(search_lst)
        df = df.rename(columns={'id': 'articleList3Id', 'name': 'articleList3Name', 'ename': 'articleList3EName'})

    if 'articleList4' in dict_name:
        df = cats.get_tv_article_list4(search_lst)
        df = df.rename(columns={'id': 'articleList4Id', 'name': 'articleList4Name', 'ename': 'articleList4EName'})
                                
    # if 'adId' in dict_name:
    #     df = cats.get_tv_ad(search_lst)
    #     df = df.rename(columns={'id': 'adId', 'name': 'adName', 'ename': 'adEName'})

    if 'adSloganAudioId' in dict_name:
        df = cats.get_tv_ad_slogan_audio(search_lst)
        df = df.rename(columns={'id': 'adSloganAudioId', 'name': 'adSloganAudioName', 'notes': 'adSloganAudioNotes'})

    if 'adSloganVideo' in dict_name:
        df = cats.get_tv_ad_slogan_video(search_lst)
        df = df.rename(columns={'id': 'adSloganVideoId', 'name': 'adSloganVideoName', 'notes': 'adSloganVideoNotes'})

    if 'region' in dict_name:
        df = cats.get_tv_region(search_lst)
        df = df.rename(columns={'id': 'regionId', 'name': 'regionName', 'ename': 'regionEName'})

    if 'tvNet' in dict_name:
        df = cats.get_tv_net()
        df = df.rename(columns={'id': 'tvNetId', 'name': 'tvNetName', 'ename': 'tvNetEName'})

    if 'tvCompany' in dict_name:
        df = cats.get_tv_company(search_lst)
        df = df.rename(columns={'id': 'tvCompanyId', 'name': 'tvCompanyName', 'ename': 'tvCompanyEName'})

    if 'adType' in dict_name:
        df = cats.get_tv_ad_type(search_lst)
        df = df.rename(columns={'id': 'adTypeId', 'name': 'adTypeName', 'ename': 'adTypeEName'})
        
    return df

In [7]:
# функция для обновления всех словарей ТВ индекс КРОМЕ nat_tv_ad_dict
# После записи данных в отчеты Simple и Buying
# в справочнике nat_tv_ad_dict - содержится cleanig_flag, который еше НЕ обновился
# таким образом у нас зафиксировано состояние с прошлой загрузки и мы можем понять, какие новые объявления появилсь в БД
# для этого по каждому отдельному столбцу, который является ключом для верхнеуровневых справочников
# мы забираем список уникальных ИД, у которых cleanig_flag=2 (т.е. только новые объявления)
# список этих ИД мы передаем в запрос к ТВ Индексу, чтобы только то, что нам нужно
# в конце жобавляем новые строки к справочникам
def update_tv_index_dicts():
    
    # сначала проверяем справочник объявлений - есть там новые или нет
    query = f"select top(1) adId from nat_tv_ad_dict where cleaning_flag=2"
    df = get_mssql_table(config.db_name, query=query)
    
    # если ответ НЕ пустой, то запускаем логику обновления всех верхнеуровневых справочников
    if not df.empty:
        # у нас сформирован справочник словарей
        # Список параметров словарей ТВ Индекс для создания таблиц в БД и нормализации данных
        # Название таблицы / Список названий полей  в БД и типы данных / Список целочисденных полей
        for key, value in config.tv_index_dicts.items():
            # передаем в SQL запрос название поля, которое нас интересует
            query = f"select distinct {key} from nat_tv_ad_dict where cleaning_flag=2"
            # отправляем запрос в Общий справочник объявлений
            ad_dict= get_mssql_table(config.db_name, query=query)
            # переводим ИД в список
            ad_dict = ad_dict[key].tolist()
            # Готовим запрос к отдельному справочнику, по котором хотим сделать обновление
            query = f"select distinct {key} from {value[0]}"
            target_dict= get_mssql_table(config.db_name, query=query)
            # переводим ИД в список
            target_dict = target_dict[key].tolist()
            
            # теперь мы хотим узнать в Общем справочнике nat_tv_ad_dict
            # есть ли какие-то ИД, которых НЕТ в другом
            search_lst = list(set(ad_dict) - set(target_dict))
            # если такие ИД есть, то передаем их на загрузку характеристик из ТВ Индекса
            if search_lst:
                # отправляем запрос в ТВ индекс
                df = get_tv_index_dicts(key, search_lst)
                # нормализуем данные
                df = normalize_columns_types(df, value[2])
                # записываем в БД
                downloadTableToDB(db_name, value[0], df)
    else:
        print(f'Новых данных для загрузки нет')

In [8]:
# update_tv_index_dicts()

Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Загрузка завершена успешно
Запрошены записи: 0 - 8
Всего найдено записей: 8

Все ок. Подключились!
Скрипт запущен 2025-04-01 18:54:46.977633
Данные добавлены в БД: mediascope_x5, таблица: tv_index_advertiser_list_dict
Скрипт отработал 2025-04-01 18:54:46.991676
Время выполнения задачи: 0:00:00.014043
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x5
##################################################

Все ок. Подключились!
Загрузка завершена успешно
Запрошены записи: 0 - 23
Всего найдено записей: 23

Все ок. Подключились!
Скрипт запущен 2025-04-01 18:54:47.090131
Данные добавлены в БД: mediascope_x5, таблица: tv_index_brand_list_dict
Скрипт отработал 2025-04-01 18:54:47.113347
Время выполнения задачи: 0:00:00.023216
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x5
##################################################

Все ок. Подключились!
Загрузка завершена успешно
Запрошены записи: 0 - 6

In [9]:
# с помощью этой функции мы создаем и заполняем словари из ТВ Индекс
# есть набор некоторы таблиц, которые НЕ связаны с ИД объявления
# соответсвенно их нет в справочнике nat_tv_ad_dict
# поэтому при первой загрузке данных мы создаем и сразу заполняем такие справочники
# в дальнейшешем их НЕ нужно обновлять, т.е. они хранятся в неизменном виде
# список справочников по умолчанию указан в этом словаре tv_index_default_dicts
def download_tv_index_default_dicts():
    for key, value in config.tv_index_default_dicts.items():
        # создаем пустые таблицы для словарей по умолчанию
        createDBTable(db_name, value[0] , value[1], flag='create')
        # отправляем запрос в ТВ индекс
        df = get_tv_index_dicts(key)
        # нормализуем данные
        df = normalize_columns_types(df, value[2])
        # записываем в БД
        downloadTableToDB(db_name, value[0], df)

In [10]:
# функция для обонвления основного справочнка объявлений nat_tv_ad_dict
# ее запускаем в самую после заливки данных из ТВ индекс и обновления всех справочников
# НО ПЕРЕД заливкой новых объявлений в гугл докс

def update_nat_tv_ad_dict():
    # забираем гугл докс с чисткой
    df_cleaning_dict = get_cleaning_dict(media_type_lst=['tv'])
    # нормализуем типы данных
    df_cleaning_dict = normalize_columns_types(df_cleaning_dict, config.custom_ad_dict_int_lst) 
    
    # создаем список из названий полей, которые нам нужны дальше для метчинга
    custom_cols_list = [col[:col.find(' ')] for col in config.custom_ad_dict_vars_list]
    custom_cols_list = list(set(custom_cols_list) - set(['ad_id', 'media_type']))
    # оставляем только нужные поля
    df_cleaning_dict = df_cleaning_dict[custom_cols_list]
    
    # формируем список названий полей, которые нам нужно забрать из БД
    # из справочника nat_tv_ad_dict
    nat_tv_ad_dict_short_cols = [col[:col.find(' ')] for col in config.nat_tv_ad_dict_vars_list]
    nat_tv_ad_dict_short_cols = list(set(nat_tv_ad_dict_short_cols) - set(custom_cols_list)) + ['media_key_id']
    # приводим список к строке
    nat_tv_ad_dict_short_cols = ', '.join(nat_tv_ad_dict_short_cols)
    
    # отправляем запрос в БД и забираем ВСЕ строки и нужные поля
    query = f"select {nat_tv_ad_dict_short_cols}  from {config.nat_tv_ad_dict}"
    nat_tv_ad_dict_df = get_mssql_table(db_name, query=query) 

    # объединяем справочник из БД с таблицей чистки
    nat_tv_ad_dict_df = nat_tv_ad_dict_df.merge(df_cleaning_dict, how='left', left_on=['media_key_id'], right_on=['media_key_id'])
    # ИД объявлений, которые НЕ нашли сопосталвения, мы считаем новыми и присваимаем им флаг=2
    nat_tv_ad_dict_df['cleaning_flag'] = nat_tv_ad_dict_df['cleaning_flag'].fillna(2)
    # остальные NaN заполняем пустотой
    nat_tv_ad_dict_df = nat_tv_ad_dict_df.fillna('')
    # создаем список полей, которые нужно оставить в этом датаФрейме
    nat_tv_ad_dict_cols = [col[:col.find(' ')] for col in config.nat_tv_ad_dict_vars_list]
    nat_tv_ad_dict_df = nat_tv_ad_dict_df[nat_tv_ad_dict_cols]
    # нормализуем типы данных
    nat_tv_ad_dict_df = normalize_columns_types(nat_tv_ad_dict_df, config.nat_tv_ad_dict_int_lst)
    
    # удаляем все данные из справочника nat_tv_ad_dict в БД 
    createDBTable(db_name, config.nat_tv_ad_dict, config.nat_tv_ad_dict_vars_list, flag='drop')
    
    # записываем новые данные в справочник Объявлений
    downloadTableToDB(db_name, config.nat_tv_ad_dict, nat_tv_ad_dict_df)

In [11]:
# update_nat_tv_ad_dict()

C:\Users\o.bogomolov\AppData\Local\Temp\2\ipykernel_60056\3050198724.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_cleaning_link, header=None, usecols=cols_count)


Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Таблица: nat_tv_ad_dict успешно удалена в БД: mediascope_x5
##########
Все ок. Подключились!
Пустая таблица nat_tv_ad_dict успешно создана в БД mediascope_x5
Все ок. Подключились!
Скрипт запущен 2025-04-01 18:54:57.161488
Данные добавлены в БД: mediascope_x5, таблица: nat_tv_ad_dict
Скрипт отработал 2025-04-01 18:54:57.197134
Время выполнения задачи: 0:00:00.035646
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x5
##################################################

